In [7]:
import os
import shutil
import random
import librosa
import math
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report
import keras
from keras.regularizers import l2
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import soundfile as sf
from pydub import AudioSegment


In [8]:
dataset_path = 'anta'
SAMPLE_RATE = 44100
N_FFT = 2048 #int(0.025 * SAMPLE_RATE)  # 25ms frame length
WIN_LENGTH = int(0.025 * SAMPLE_RATE)
HOP_LENGTH = 512 #int(0.010 * SAMPLE_RATE)     # 10ms hop length
N_MFCC = 20  # Number of cepstral coefficients to be extracted
N_MELS =40
DURATION = 30  # Duration of each audio file
NUM_SEGMENTS = 10  # Number of segments per audio file
SAMPLES_PER_SEGMENT = DURATION * SAMPLE_RATE


In [9]:
import os

# List all directories in dataset_path
directories = os.listdir(dataset_path)
jumlah = 0

# Loop through each directory and count the number of files in it
for directory in directories:
    path = os.path.join(dataset_path, directory)
    if os.path.isdir(path):
        num_files = len(os.listdir(path))
        jumlah += num_files
        print(f"{directory}: {num_files} songs")

print(f"Total: {jumlah} songs")


classical: 320 songs
electronic: 115 songs
jazz: 26 songs
metal: 29 songs
pop: 6 songs
punk: 16 songs
rock: 95 songs
world: 122 songs
Total: 729 songs


In [14]:
preprocess = {
    "mapping" : [],
    "train" : {
        "features" : [],
        "labels" : []
    },
    "test" : {
        "features" : [],
        "labels" : []
    },
    "val" : {
        "features" : [],
        "labels" : []
    }
}

In [11]:
def process(audio):
    # Set amplitude threshold
    amplitude_threshold = 0.00316 # -50dB

    # Create an array of silent frames based on amplitude threshold
    silent_frames = np.abs(audio) < amplitude_threshold

    # Create an array of non-silent frames by inverting the silent frames
    non_silent_frames = np.logical_not(silent_frames)

    # Extract non-silent audio frames
    non_silent_audio = audio[non_silent_frames]

    stretch_factor = len(non_silent_audio) / (sr * 60)

    duration = librosa.get_duration(y=audio, sr=sr)
    if duration < 60:
        audio = librosa.effects.time_stretch(y=audio, rate=stretch_factor)
    
    # ambil detik ke 30 sampai 60
    audio = audio[SAMPLES_PER_SEGMENT:(SAMPLES_PER_SEGMENT*2)]
    

    return audio



In [12]:
def slice_per_segment(audio, num_segment) :
    dummy = []
    # Menghitung durasi total audio dalam milidetik
    audio = process(audio)

    # Menghitung durasi per segment
    segment_duration = int(SAMPLES_PER_SEGMENT / num_segment)

    for s in range(num_segment) :
        start_sample = segment_duration * s
        end_sample = start_sample + segment_duration
        segment_audio = audio[start_sample:end_sample]
        print(librosa.get_duration(y=segment_audio, sr=sr))
        dummy.append(segment_audio)

    return dummy

In [15]:
for i, (dirpath, dirnames, filenames) in enumerate(os.walk(dataset_path)):

    # Pastikan kita tidak berada di root level
    if dirpath is not dataset_path:
        # Simpan nama genre ke mapping
        dirpath_components = dirpath.split("/")
        semantic_label = dirpath_components[-1]
        preprocess["mapping"].append(semantic_label)
        print("\nProcessing {}".format(semantic_label))

        feature_dummies = []
        label_dummies = []

        # Proses file audio untuk tiap genre
        for f in filenames:
            # Load audio file
            file_path = os.path.join(dirpath, f)
            audio, sr = librosa.load(file_path, sr=SAMPLE_RATE)
            processed = slice_per_segment(audio, NUM_SEGMENTS)
            for s,a in enumerate(processed) :
                feature_dummies.append(a)
                label_dummies.append(i-1)
                print("{}, segment:{}".format(file_path, s+1))

        # Split dataset menjadi training, validation, dan test set dengan train_test_split
        feature_train, feature_temp, label_train, label_temp = train_test_split(feature_dummies, label_dummies, test_size=0.2, random_state=42)
        feature_test, feature_val, label_test, label_val = train_test_split(feature_temp, label_temp, test_size=0.5, random_state=42)

        # Menambahkan data ke dalam dictionary
        preprocess["train"]["features"].extend(feature_train)
        preprocess["train"]["labels"].extend(label_train)
        preprocess["test"]["features"].extend(feature_test)
        preprocess["test"]["labels"].extend(label_test)
        preprocess["val"]["features"].extend(feature_val)
        preprocess["val"]["labels"].extend(label_val)



Processing anta\classical
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
anta\classical\1-concerto_no.1_in_d_major_rv_2.wav, segment:1
anta\classical\1-concerto_no.1_in_d_major_rv_2.wav, segment:2
anta\classical\1-concerto_no.1_in_d_major_rv_2.wav, segment:3
anta\classical\1-concerto_no.1_in_d_major_rv_2.wav, segment:4
anta\classical\1-concerto_no.1_in_d_major_rv_2.wav, segment:5
anta\classical\1-concerto_no.1_in_d_major_rv_2.wav, segment:6
anta\classical\1-concerto_no.1_in_d_major_rv_2.wav, segment:7
anta\classical\1-concerto_no.1_in_d_major_rv_2.wav, segment:8
anta\classical\1-concerto_no.1_in_d_major_rv_2.wav, segment:9
anta\classical\1-concerto_no.1_in_d_major_rv_2.wav, segment:10
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
anta\classical\1-handel_trio_sonata_op._5_no.4.wav, segment:1
anta\classical\1-handel_trio_sonata_op._5_no.4.wav, segment:2
anta\classical\1-handel_trio_sonata_op._5_no.4.wav, segment:3
anta\classical\1-handel_trio_sonata_op._5_no.4.wav, segment:4
anta\classical\1-handel_

In [ ]:
import pickle

# Menyimpan objek preprocess ke dalam file preprocess.pkl
with open('preprocess.pkl', 'wb') as f:
    pickle.dump(preprocess, f)


In [ ]:
# Load data from pickle file
with open("preprocess.pkl", "rb") as f:
    data = pickle.load(f)

In [ ]:
print(data["mapping"])

['anta\\classical', 'anta\\electronic', 'anta\\jazz', 'anta\\metal', 'anta\\pop', 'anta\\punk', 'anta\\rock', 'anta\\world']


In [ ]:
print(np.array(preprocess["train"]["features"]).shape)
print(np.array(preprocess["train"]["labels"]).shape)
print(np.array(preprocess["test"]["features"]).shape)
print(np.array(preprocess["test"]["labels"]).shape)
print(np.array(preprocess["val"]["features"]).shape)
print(np.array(preprocess["val"]["labels"]).shape)


(5832, 66150)
(5832,)
(729, 66150)
(729,)
(729, 66150)
(729,)


In [ ]:
# For change format music file
counter = 0
# Daftar genre dan path ke direktori sumber lagu
genres = ['classical','electronic','jazz','metal','pop','punk','rock','world']
source_directory = "dataset_ismir"

# Path ke direktori utama tempat dataset akan dibuat
dataset_directory = "F:\\anta"

# Loop melalui setiap genre
for genre in genres:
    # Membuat direktori genre di dalam dataset_directory
    genre_directory = os.path.join(dataset_directory, genre)
    # print(genre_directory)
    os.makedirs(genre_directory, exist_ok=True)

    # Path ke direktori sumber lagu untuk genre ini
    genre_source_directory = os.path.join(source_directory, genre)
    print(genre_source_directory)

    # Loop melalui setiap file lagu
    for song in os.listdir(genre_source_directory) :
        file_path = os.path.join(genre_source_directory, song)
        # Baca file MP3
        audio = AudioSegment.from_mp3(file_path)

        # Konversi ke format WAV
        output_filename = os.path.splitext(song)[0] + ".wav"
        output_path = os.path.join(genre_directory, output_filename)
        audio.export(output_path, format="wav")
        print("Processing {}".format(output_filename)) 
        counter += 1
                

dataset_ismir\classical
Processing 1-concerto_no.1_in_d_major_rv_2.wav
Processing 1-handel_trio_sonata_op._5_no.4.wav
Processing 1-matteis_ground_after_the_scotch_humour.wav
Processing 1-ma_belle_si_ton_ame_(gilles_durant_de_la_bergerie).wav
Processing 1-oxenford_(anon).wav
Processing 1-pandolfi_la_berbabea_op_4_#1.wav
Processing 1-prelude_(phalese).wav
Processing 1-preludio_i.wav
Processing 1-recercar_1.wav
Processing 1-set_a_6_in_c_major.wav
Processing 1-suite_iii_in_do_maggiore_-_prelude.wav
Processing 1-to_thee_we_sing_bulgarian_chant_tone_legetos.wav
Processing 1-we_are_guarded_by_the_cross.wav
Processing 1-widmung.wav
Processing 10-gigue.wav
Processing 10-guillarde_milanoise_(morlaye).wav
Processing 10-i_care_not_for_these_ladies_(thomas_campian).wav
Processing 10-johann_christian_bach_(arr_mozart)_concerto_in_d_k107_(1772).wav
Processing 10-partita_prima_in_mi_minore_-_double.wav
Processing 10-recercar_10_(steel_string_guitar).wav
Processing 10-sonata_iii_moderato.wav
Processing

In [ ]:
print(counter)

729
